In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import models
import pandas as pd
import mscthesis
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go

In [51]:
song_visits = models.VisitsFromFile("./../../dbs/sweden/visits-zipf1.2.csv").visits()
song_visits.shape

(2400181, 6)

In [94]:
raw_visits = models.VisitsFromGeotweetsFile("./../../dbs/sweden/geotweets.csv").visits()
raw_visits.shape

(1292641, 7)

In [95]:
raw_visits.head(5)

,region,latitude,longitude,day,timeslot,kind,label
userid,,,,,,,
5616,0,57.599221,18.436371,1694,17,region,other
5616,0,57.599221,18.436371,1768,11,region,other
5616,0,57.599221,18.436371,1924,15,region,other
5616,0,57.599221,18.436371,2139,20,region,other
5616,0,57.599221,18.436371,2155,8,region,other


In [89]:
USERID = 392123

In [59]:
song_user = song_visits.loc[USERID].reset_index().sort_values(['userid', 'day','timeslot']).set_index('userid')

In [97]:
raw_user = raw_visits.loc[USERID].reset_index().sort_values(['userid', 'day','timeslot']).set_index('userid')

In [111]:
song_user = mscthesis.remove_consecutive_region_visits(song_user)
song_user.shape

(503, 6)

In [112]:
raw_user = mscthesis.remove_consecutive_region_visits(raw_user)
raw_user.shape

(452, 7)

In [140]:
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = song_user['longitude'],
    lat = song_user['latitude'],
    marker = {'size': 5},
    text=song_user,
    name="Song"
))

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = raw_user['longitude'],
    lat = raw_user['latitude'],
    marker = {'size': 5},
    hovertext=raw_user,
    name="Raw"
))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'style': "stamen-terrain"
    })

fig.show()

In [139]:
trace1 = go.Histogram(
    x=song_user['region'],
    opacity=0.6,
    name='Song',
    xbins=dict(
        start=-1.0,
        end=raw_user['region'].max(),
        size=1
    ),
    histnorm='percent'
)
trace2 = go.Histogram(
    x=raw_user['region'],
    opacity=0.6,
    name = 'Raw',
    xbins=dict(
        start=-1.0,
        end=raw_user['region'].max(),
        size=1
    ),
    histnorm='percent'
)
data = [trace1, trace2]
layout = go.Layout(
    title='Region distribution User: ' + str(USERID),
    xaxis=dict(
        title='Region'
    ),
    yaxis=dict(
        title='Percent of trips'
    ),
    barmode='overlay'
)
fig = go.Figure(data=data, layout=layout)
#fig.show()

In [110]:
#Home region
raw_user[raw_user['label'] == 'home'].iloc[0]['region']

22

In [128]:
song_utrips = mscthesis.visit_gaps(song_visits)
raw_utrips = mscthesis.visit_gaps(raw_visits)

In [129]:
song_utrips['distance'] = song_utrips.apply(lambda x: mscthesis.haversine_distance(x['latitude_origin'],x['longitude_origin'],x['latitude_destination'],x['longitude_destination']), axis=1)
song_utrips.head(5)


,day_origin,timeslot_origin,kind_origin,latitude_origin,longitude_origin,region_origin,day_destination,timeslot_destination,kind_destination,latitude_destination,longitude_destination,region_destination,distance
userid,,,,,,,,,,,,,
5616,0,0,region,59.426889,17.954336,5,0,1,region,59.405498,17.954750,6,2.379385
5616,0,1,region,59.405498,17.954750,6,0,2,region,57.398813,18.874708,52,229.546431
5616,0,2,region,57.398813,18.874708,52,0,3,region,59.405498,17.954750,6,229.546431
5616,0,3,region,59.405498,17.954750,6,0,4,region,59.445959,17.942841,347,4.550543
5616,0,4,region,59.445959,17.942841,347,1,0,region,59.426889,17.954336,5,2.218523


In [130]:
raw_utrips['distance'] = raw_utrips.apply(lambda x: mscthesis.haversine_distance(x['latitude_origin'],x['longitude_origin'],x['latitude_destination'],x['longitude_destination']), axis=1)
raw_utrips.head(5)


,region_origin,latitude_origin,longitude_origin,day_origin,timeslot_origin,kind_origin,label_origin,region_destination,latitude_destination,longitude_destination,day_destination,timeslot_destination,kind_destination,label_destination,distance
userid,,,,,,,,,,,,,,,
5616,0,57.599221,18.436371,1694,17,region,other,0,57.599221,18.436371,1768,11,region,other,0.0
5616,0,57.599221,18.436371,1768,11,region,other,0,57.599221,18.436371,1924,15,region,other,0.0
5616,0,57.599221,18.436371,1924,15,region,other,0,57.599221,18.436371,2139,20,region,other,0.0
5616,0,57.599221,18.436371,2139,20,region,other,0,57.599221,18.436371,2155,8,region,other,0.0
5616,0,57.599221,18.436371,2155,8,region,other,0,57.599221,18.436371,2191,20,region,other,0.0


In [135]:
trace1 = go.Histogram(
    x=song_utrips['distance'],
    opacity=0.6,
    name='Song',
    xbins=dict(
        start=0.0,
        end=raw_utrips['distance'].max(),
        size=5
    ),
    histnorm='percent'
)
trace2 = go.Histogram(
    x=raw_utrips['distance'],
    opacity=0.6,
    name = 'Raw',
    xbins=dict(
        start=0.0,
        end=raw_utrips['distance'].max(),
        size=5
    ),
    histnorm='percent'
)
data = [trace1, trace2]
layout = go.Layout(
    title='Trip distance distribution',
    xaxis=dict(
        title='Distance (km)'
    ),
    yaxis=dict(
        title='Percent of trips'
    ),
    barmode='overlay'
)
fig = go.Figure(data=data, layout=layout)
#fig.show()